# Ad click 2

### Import

In [1]:
import numpy as np
import pandas as pd
from sklearn.svm import SVC ,SVR ,LinearSVC
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

###  Load

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [3]:
train.shape, test.shape

((12137810, 10), (3706907, 9))

### Converting Datetime to hours time

#### Train data

In [4]:
time_list_temp = []
for time in train['datetime']:
    time_list_temp.append(time)

In [5]:
from datetime import datetime
time_list = []

In [6]:
for i in time_list_temp:
    datetime_object = datetime.strptime(i, '%Y-%m-%d %H:%M:%S')
    time_list.append(datetime_object.hour)

In [7]:
train['datetime'] = time_list

#### Test data

In [8]:
time_list_temp = []
for time in test['datetime']:
    time_list_temp.append(time)
time_list = []
for i in time_list_temp:
    datetime_object = datetime.strptime(i, '%Y-%m-%d %H:%M:%S')
    time_list.append(datetime_object.hour)
test['datetime'] = time_list

### Rename

In [9]:
train.rename(index=str, columns={"datetime":'time'},inplace = True)
train.head(1)

,ID,time,siteid,offerid,category,merchant,countrycode,browserid,devid,click
0,IDsrk7SoW,9,4709696.0,887235,17714,20301556,e,Firefox,NaN,0


In [10]:
test.rename(index=str, columns={"datetime":'time'},inplace = True)

### Stats (Click ratio)

In [11]:
train[['time','click']].groupby(['time']).mean().sort_values(by='click', ascending=False)

,click
time,
21,0.477524
22,0.473212
20,0.467624
23,0.432123
0,0.376891
1,0.336403
19,0.114185
18,0.046388
7,0.038357


Highest click rate is between 8-12 i.e (43%+)

In [12]:
train[['countrycode','click']].groupby(['countrycode']).mean().sort_values(by='click', ascending=False)

,click
countrycode,
c,0.271424
d,0.183294
e,0.018513
f,0.018384
a,0.007335
b,0.004134


Highest click ratio is from country code-c followed by d.

In [13]:
train[['devid','click']].groupby(['devid']).mean().sort_values(by='click', ascending=False)

,click
devid,
Desktop,0.067059
Mobile,0.039498
Tablet,0.005512


Highest click ratio is from Desktop.

In [14]:
train[['browserid','click']].groupby(['browserid']).mean().sort_values(by='click', ascending=False)

,click
browserid,
InternetExplorer,0.251632
Google Chrome,0.207429
Safari,0.036605
Chrome,0.036323
Opera,0.036053
IE,0.035961
Internet Explorer,0.035513
Mozilla Firefox,0.008221
Mozilla,0.007177


InternetExplorer and Google Chrome are most widely used browsers.

In [15]:
click_true_train = train[train['click']==1] 

In [16]:
click_true_train.head()

,ID,time,siteid,offerid,category,merchant,countrycode,browserid,devid,click
13,IDdPZWVCs,21,6718234.0,433512,43612,57517812,c,NaN,Desktop,1
16,IDU8ukCsv,15,469603.0,385750,93286,7122654,a,Google Chrome,Mobile,1
35,IDfEQxvi3,19,5369414.0,513860,27655,92826840,f,Google Chrome,Mobile,1
67,IDbF2aKjs,19,6005717.0,956737,1678,60293830,f,InternetExplorer,Desktop,1
90,IDEgpeME8,14,7979331.0,501647,68947,58321067,f,Mozilla,Desktop,1


In [17]:
(test.isnull().sum()/train.shape[0] ) *100

ID             0.000000
time           0.000000
siteid         3.054719
offerid        0.000000
category       0.000000
merchant       0.000000
countrycode    0.000000
browserid      1.828221
devid          5.805158
dtype: float64

since null values are very less in click_true data so even if we drop rows with na there would be no significant effect.

In [18]:
click_true_train = train[train['click']==1]

In [19]:
len(click_true_train)/len(train)*100

3.6020830775897794

In [20]:
clicks_counts = train['click'].value_counts()
clicks_counts[1]/(clicks_counts[0]+clicks_counts[1])*100

3.6020830775897794

only 3.599 % of total ads displayed get clicked.

### Cleaning train data

In [21]:
(train.isnull().sum()/train.shape[0] ) *100

ID              0.000000
time            0.000000
siteid          9.989570
offerid         0.000000
category        0.000000
merchant        0.000000
countrycode     0.000000
browserid       5.011835
devid          14.996931
click           0.000000
dtype: float64

In [22]:
print(train.shape[0])

12137810


In [23]:
train.dropna(subset = ['devid','siteid','browserid'],inplace = True)
print(train.shape[0])

8821613


In [24]:
8821613/12137810

0.7267878637085273

now we have only 72% data left with which we will train.

### Cleaning test data

In [25]:
(test.isnull().sum()/train.shape[0] ) *100

ID             0.000000
time           0.000000
siteid         4.203041
offerid        0.000000
category       0.000000
merchant       0.000000
countrycode    0.000000
browserid      2.515481
devid          7.987417
dtype: float64

####  Get max values

In [26]:
site_id_count = test['siteid'].value_counts()
s = pd.DataFrame(site_id_count)
max_siteid =s.index[0]
max_siteid

3696590.0

In [27]:
b_id_count = test['browserid'].value_counts()
b = pd.DataFrame(b_id_count)
max_browseid =b.index[0]
max_browseid

'Edge'

In [28]:
d_id_count = test['devid'].value_counts()
d = pd.DataFrame(d_id_count)
max_devid =d.index[0]
max_devid

'Mobile'

####  Replace with max values.

In [29]:
test['siteid'].fillna(max_siteid,inplace = True)
test['browserid'].fillna(max_browseid,inplace = True)
test['devid'].fillna(max_devid,inplace = True)

Now both train and test data are completely clean.

### Handling non-numerical values.

In [30]:
def handle_non_numerical_data(df):
    columns = df.columns.values
    for column in columns:
        text_digit_vals = {}
        
        if df[column].dtype != np.int64 and df[column].dtype != np.float64:
            column_content = df[column].values.tolist()
            unique_elements = set(column_content)
            #print(unique_elements)
            x = 0
            for unique in unique_elements:
                if unique not in text_digit_vals:
                    text_digit_vals[unique] = x
                    x+=1
            #print(text_digit_vals) ## ex:{'male': 0, 'female': 1}
                
            def convert_to_int(key):
                return text_digit_vals[key]
            df[column] = list(map(convert_to_int, df[column]))

In [31]:
handle_non_numerical_data(train)

In [32]:
handle_non_numerical_data(test)

### Creating logits and labels

In [33]:
train.head(1)

,ID,time,siteid,offerid,category,merchant,countrycode,browserid,devid,click
1,3994496,17,5189467.0,178235,21407,9434818,0,4,2,0


In [34]:
train_X = train.drop(['ID','click'],axis = 1)
train_y = train['click']
test_X  = test.drop(['ID'],axis = 1)
test_ids = test['ID']

### create a small training data for quick testing 

In [35]:
x_train_s,x_test_s,y_train_s,y_test_s = train_test_split(train_X,train_y, train_size = 0.0025)

c:\python36\lib\site-packages\sklearn\model_selection\_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [36]:
x_train_ss,x_test_ss,y_train_ss,y_test_ss = train_test_split(x_test_s,y_test_s, train_size = 0.0025)

c:\python36\lib\site-packages\sklearn\model_selection\_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [37]:
x_train_ss.shape,y_train_ss.shape

((21998, 8), (21998,))

In [38]:
train_X.shape,x_train_s.shape,y_train_s.shape

((8821613, 8), (22054, 8), (22054,))

In [39]:
clfs  = [SVC(),
         SVR(),
         LinearSVC(),
         RandomForestClassifier(),
         GradientBoostingClassifier(),
         LogisticRegression(),
         KNeighborsClassifier(),
         GaussianNB(),
         Perceptron(),
         SGDClassifier(),
         DecisionTreeClassifier()]

####  Train accuracy

In [40]:
for clf in clfs:
    clf.fit(x_train_s,y_train_s)
    print(clf.score(x_train_s,y_train_s)) 

1.0
0.7276244706415944
0.9592817629454974
0.9972340618481909
0.9813639249115806
0.961820984855355
0.962093044345697
0.961820984855355
0.961820984855355
0.961820984855355
1.0


c:\python36\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.perceptron.Perceptron'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
c:\python36\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


#### Testing Accuracy

In [41]:
for clf in clfs:
    print(clf.score(x_train_ss,y_train_ss)) 

0.964314937721611
-0.2541421731002651
0.9619965451404673
0.9772252022911174
0.9794981361941995
0.9642240203654878
0.9633603054823165
0.9642240203654878
0.9642240203654878
0.9642240203654878
0.9664060369124465


# Training

In [60]:
clf =  GradientBoostingClassifier()
clf.fit(x_train_s,y_train_s)
print(clf.score(x_train_ss,y_train_ss)) 

0.9794981361941995


## Prediction

In [61]:
clf =  GradientBoostingClassifier()
clf.fit(train_X,train_y)
sub = clf.predict(test_X)
submission = pd.DataFrame({'ID':test_ids,'click':sub})
submission.to_csv('submissionClick2gcb.csv',index = False)
submission.head() #12:05

,ID,click
0,1823273,0
1,1984229,0
2,1502902,0
3,1330380,0
4,1704843,0


In [62]:
datetime.now()

datetime.datetime(2018, 5, 29, 0, 29, 7, 990450)

In [63]:
sub = pd.read_csv('submissionClick2gcb.csv')

In [64]:
items_counts = sub['click'].value_counts()
max_item = items_counts.max()

In [66]:
items_counts

0    3514555
1     192352
Name: click, dtype: int64

In [70]:
295254/3411653*100

8.65427990478516

In [69]:
192352/3514555*100

5.473011519239278